In [1]:
from transformers import TFAutoModelForCausalLM, AutoTokenizer, DefaultDataCollator, Trainer, TrainingArguments, BertTokenizerFast, BertForQuestionAnswering
from datasets import Dataset, DatasetDict, load_dataset
# import tensorflow as tf
# import plotly.express as px
# import plotly.io as pio
# import pandas as pd
# import math
# import os, io

2023-04-01 10:15:46.009868: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
tokenizer = BertTokenizerFast.from_pretrained("distilbert-base-uncased")
model = BertForQuestionAnswering.from_pretrained("distilbert-base-uncased")
# tokenizer.pad_token = tokenizer.eos_token

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DistilBertTokenizer'. 
The class this function is called from is 'BertTokenizerFast'.
You are using a model of type distilbert to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.
Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing BertForQuestionAnswering: ['distilbert.transformer.layer.0.attention.k_lin.weight', 'distilbert.transformer.layer.4.output_layer_norm.bias', 'distilbert.transformer.layer.2.attention.q_lin.bias', 'distilbert.transformer.layer.5.attention.v_lin.bias', 'distilbert.transformer.layer.5.attention.v_lin.weight', 'distilbert.transformer.layer.2.ffn.lin1.weight', 'distilbert.transformer.layer.5.ffn.lin2.bias', 'distilbert.transformer.layer.0.output_layer_n

In [3]:
data = load_dataset("./source/train.json", split='train')
data[0]

Found cached dataset csv (C:/Users/tom/.cache/huggingface/datasets/CShorten___csv/CShorten--ML-ArXiv-Papers-0dcddd7fc76c9211/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)


Dataset({
    features: ['Unnamed: 0.1', 'Unnamed: 0', 'title', 'abstract'],
    num_rows: 117592
})

In [4]:
data = data.train_test_split(shuffle = True, seed = 200, test_size=0.2)

train = data["train"]
val = data["test"]

Loading cached split indices for dataset at C:\Users\tom\.cache\huggingface\datasets\CShorten___csv\CShorten--ML-ArXiv-Papers-0dcddd7fc76c9211\0.0.0\6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317\cache-5fcced655579dd43.arrow and C:\Users\tom\.cache\huggingface\datasets\CShorten___csv\CShorten--ML-ArXiv-Papers-0dcddd7fc76c9211\0.0.0\6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317\cache-10bb285727aaa68b.arrow


In [5]:
# The tokenization function
def tokenization(data):
    tokens = tokenizer(data["abstract"], padding="max_length", truncation=True, max_length=300)
    return tokens

# Apply the tokenizer in batch mode and drop all the columns except the tokenization result
train_token = train.map(tokenization, batched = True, remove_columns=["title", "abstract", "Unnamed: 0", "Unnamed: 0.1"], num_proc=10)
val_token = val.map(tokenization, batched = True, remove_columns=["title", "abstract", "Unnamed: 0", "Unnamed: 0.1"], num_proc=10)

Map (num_proc=10):   0%|          | 0/94073 [00:00<?, ? examples/s]

NameError: name 'tokenizer' is not defined

In [ ]:
# Create labels as a copy of input_ids
def create_labels(text):
    text["labels"] = text["input_ids"].copy()
    return text

# Add the labels column using map()
lm_train = train_token.map(create_labels, batched=True, num_proc=10)
lm_val = val_token.map(create_labels, batched=True, num_proc=10)